In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp client

<IPython.core.display.Javascript object>

In [ ]:
# export

import os
import time
import httpx
import asyncio
import hashlib
import aiohttp

from pathlib import Path

from will_it_saturate.core import Benchmark

<IPython.core.display.Javascript object>

In [ ]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte

benchmark = Benchmark(bandwidth=bandwidth, duration=3)

# file_sizes = [10 ** 7, 10 ** 6]
file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
# file_sizes = [10 ** 7]
benchmark.create_rows(file_sizes)

<IPython.core.display.Javascript object>

In [ ]:
# export


class BaseClient:
    def __init__(self, benchmark):
        self.benchmark = benchmark

    def check_md5sums(self, benchmark_files, responses):
        md5_lookup = {}
        for response in responses:
            url = str(response.url)
            md5_lookup[url] = hashlib.md5(response.content).hexdigest()

        for bf in benchmark_files:
            assert bf.md5sum == md5_lookup.get(bf.url, "wrong")

<IPython.core.display.Javascript object>

In [ ]:
# export


class HttpxClient(BaseClient):
    async def measure_benchmark_row(self, br):
        urls = [bf.url for bf in br.files]
        # httpx breaks on more than 100 parallel connections
        max_connections = min(br.number_of_connections, 100)
        limits = httpx.Limits(
            max_keepalive_connections=5, max_connections=max_connections
        )
        start = time.perf_counter()
        async with httpx.AsyncClient(limits=limits) as client:
            responses = await asyncio.gather(*[client.get(url) for url in urls])
        elapsed = time.perf_counter() - start
        self.check_md5sums(br.files, responses)
        br.elapsed = elapsed

    async def run(self):
        for br in self.benchmark.rows:
            await self.measure_benchmark_row(br)

<IPython.core.display.Javascript object>

In [ ]:
bclient = HttpxClient(benchmark)

<IPython.core.display.Javascript object>

In [ ]:
await bclient.run()

<IPython.core.display.Javascript object>

In [ ]:
for br in benchmark.rows:
    print(br.bytes_per_second)

115508622.77644457
92834362.10747789
55720874.5777953


<IPython.core.display.Javascript object>

# aiohttp

In [ ]:
# export


class AioHttpResponse:
    def __init__(self, url, content):
        self.url = url
        self.content = content


class AioHttpClient(BaseClient):
    async def fetch_page(self, session, url):
        async with session.get(url) as response:
            content = await response.read()
            return AioHttpResponse(url, content)

    async def measure_benchmark_row(self, br):
        urls = [bf.url for bf in br.files]
        max_connections = min(br.number_of_connections, 300)
        conn = aiohttp.TCPConnector(limit=max_connections)
        responses = []
        start = time.perf_counter()
        async with aiohttp.ClientSession(connector=conn) as session:
            tasks = [asyncio.create_task(self.fetch_page(session, url)) for url in urls]
            responses = await asyncio.gather(*tasks)
        elapsed = time.perf_counter() - start
        self.responses = responses
        self.check_md5sums(br.files, responses)
        br.elapsed = elapsed

    async def run(self):
        for br in self.benchmark.rows:
            await self.measure_benchmark_row(br)

<IPython.core.display.Javascript object>

In [ ]:
bclient = AioHttpClient(benchmark)

<IPython.core.display.Javascript object>

In [ ]:
await bclient.run()

<IPython.core.display.Javascript object>

In [ ]:
for br in benchmark.rows:
    print(br.elapsed)
    print(br.bytes_per_second)

3.223404542000001
116336623.31670187
3.539465290999999
105948206.62701057
5.533927666999997
67763805.84737416


<IPython.core.display.Javascript object>

# Export

In [ ]:
from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted 01_serve_files.ipynb.
Converted 02_run_benchmark.ipynb.
Converted 03_create_files_old.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>